<a href="https://colab.research.google.com/github/KJHHHHHHH123/20240322git/blob/data_processimg/KMA_API_Test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
import datetime

# 기본 API URL과 인증 키 설정
base_url = "https://apihub.kma.go.kr/api/typ01/url/kma_sfctm3.php"
auth_key = "wnNFMnRfQYCzRTJ0X3GATw"

# 시작 및 종료 날짜 설정
start_date = datetime.datetime(2020, 1, 14)
end_date = datetime.datetime(2020, 6, 14)

# 결과를 저장할 리스트
all_data = []

# 월별로 데이터를 요청
current_date = start_date
while current_date < end_date:
    next_month = current_date + datetime.timedelta(days=30)  # 다음 달로 이동
    if next_month > end_date:
        next_month = end_date  # 마지막 범위 설정

    # 요청 URL 생성
    url = f"{base_url}?tm1={current_date.strftime('%Y%m%d%H%M')}&tm2={next_month.strftime('%Y%m%d%H%M')}&stn=108&authKey={auth_key}"
    print(f"Requesting data from {current_date} to {next_month}")

    # API 요청
    response = requests.get(url)

    # JSON 형식인지 확인 후 데이터 추가
    if 'application/json' in response.headers.get('Content-Type', ''):
        json_response = response.json()
        all_data.append(json_response)
    else:
        print("Response is not in JSON format. Content-Type:", response.headers.get('Content-Type'))
        print("Response content:", response.text)

    # 현재 날짜를 다음 달로 갱신
    current_date = next_month

# 결과 출력
print("모든 데이터를 성공적으로 가져왔습니다!")
print(all_data)


Requesting data from 2020-01-14 00:00:00 to 2020-02-13 00:00:00
Response is not in JSON format. Content-Type: text/plain;charset=EUC-KR
Response content: #START7777
#2345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234
# YYMMDDHHMI STN  WD   WS GST  GST  GST     PA     PS PT    PR    TA    TD    HM    PV     RN     RN     RN     RN     SD     SD     SD WC WP WW                      CA  CA   CH CT        CT  CT  CT    VS   SS    SI ST    TS    TE    TE    TE    TE  ST   WH BF IR IX
#        KST  ID  16  m/s  WD   WS   TM    hPa    hPa  -   hPa     C     C     %   hPa     mm    DAY    JUN    INT    HR3    DAY    TOT -- -- ---------------------- TOT MID  MIN -------- TOP MID LOW                  GD     C     5    10    20    30 SEA    m --  -  -
202001140000 108  34  0.5  -9 -9.0   -9 1014.5

In [14]:
import requests
import datetime

# 기본 API URL과 인증 키 설정
base_url = "https://apihub.kma.go.kr/api/typ01/url/kma_sfctm3.php"
auth_key = "wnNFMnRfQYCzRTJ0X3GATw"

# 시작 및 종료 날짜 설정
start_date = datetime.datetime(2020, 1, 14)
end_date = datetime.datetime(2020, 6, 14)

# 월별로 데이터를 요청
current_date = start_date
all_text = ""

while current_date < end_date:
    next_month = current_date + datetime.timedelta(days=30)
    if next_month > end_date:
        next_month = end_date

    url = f"{base_url}?tm1={current_date.strftime('%Y%m%d%H%M')}&tm2={next_month.strftime('%Y%m%d%H%M')}&stn=108&authKey={auth_key}"
    print(f"Requesting data from {current_date} to {next_month}")

    response = requests.get(url)

    if 'application/json' in response.headers.get('Content-Type', ''):
        pass
    else:
        all_text += response.text + "\n"

    current_date = next_month

lines = all_text.splitlines()

# 필요한 컬럼명 수정
target_columns = ["STN", "TA", "RN", "INT", "DAY", "WW", "WC", "WP", "HR3", "TOT", "TOP", "LOW", "WH", "WD", "HM"]

# 헤더 라인 찾기
header_line1 = None
header_line2 = None
for i, line in enumerate(lines):
    if line.startswith('# YYMMDDHHMI'):
        header_line1 = line
        header_line2 = lines[i+1]
        break

if not header_line1 or not header_line2:
    print("헤더 라인을 찾지 못했습니다.")
    exit()

# 헤더 라인 파싱
header_cols1 = header_line1.strip('# ').split()
header_cols2 = header_line2.strip('# ').split()

# 컬럼 인덱스 매핑 초기화
col_index_map = {}

# 특수 컬럼들의 위치 찾기
for i, (col1, col2) in enumerate(zip(header_cols1, header_cols2)):
    # CA TOT 찾기
    if col1 == "CA" and col2 == "TOT":
        col_index_map["TOT"] = i
    # CT TOP 찾기
    elif col1 == "CT" and col2 == "TOP":
        col_index_map["TOP"] = i
    # CT LOW 찾기
    elif col1 == "CT" and col2 == "LOW":
        col_index_map["LOW"] = i
    # RN INT 찾기
    elif col1 == "RN" and col2 == "INT":
        col_index_map["INT"] = i
    # RN DAY 찾기
    elif col1 == "RN" and col2 == "DAY":
        col_index_map["DAY"] = i
    # SD HR3 찾기
    elif col1 == "SD" and col2 == "HR3":
        col_index_map["HR3"] = i
    # SD DAY 찾기
    elif col1 == "SD" and col2 == "DAY":
        col_index_map["SD_DAY"] = i
    # SD TOT 찾기
    elif col1 == "SD" and col2 == "TOT":
        col_index_map["SD_TOT"] = i

# 나머지 일반 컬럼들의 인덱스 찾기
for col in ["STN", "TA", "RN", "WW", "WC", "WP", "WH", "WD", "WS", "HM"]:
    if col in header_cols1:
        col_index_map[col] = header_cols1.index(col)

# 컬럼 헤더 정의
columns = ["DateTime(YYYYMMDDHHMI)", "STN", "TA", "RN", "RN_INT", "RN_DAY", "WW", "WC", "WP",
           "SD_DAY", "SD_TOT", "SD_HR3", "CA_TOT", "CT_TOP", "CT_LOW", "WH", "WD", "WS", "HM"]

# 각 컬럼의 너비 설정
widths = {
    "DateTime(YYYYMMDDHHMI)": 20,
    "STN": 6,
    "TA": 8,
    "RN": 8,
    "RN_INT": 8,
    "RN_DAY": 8,
    "WW": 6,
    "WC": 6,
    "WP": 6,
    "SD_DAY": 8,
    "SD_TOT": 8,
    "SD_HR3": 8,
    "CA_TOT": 8,
    "CT_TOP": 8,
    "CT_LOW": 8,
    "WH": 8,
    "WD": 6,
    "WS": 6,
    "HM": 8
}

# 헤더 출력
header = " ".join(f"{col:^{widths[col]}}" for col in columns)
print(header)
print("-" * len(header))  # 구분선 추가

# 데이터 출력
for line in lines:
    if line.startswith('#'):
        continue
    parts = line.split()
    if len(parts) > 1 and parts[0].isdigit() and len(parts[0]) == 12:
        dt = parts[0]
        values = [dt]

        columns_order = ["STN", "TA", "RN", "INT", "DAY", "WW", "WC", "WP",
                        "SD_DAY", "SD_TOT", "HR3", "TOT", "TOP", "LOW", "WH", "WD", "WS", "HM"]

        for col in columns_order:
            if col in col_index_map:
                values.append(parts[col_index_map[col]])
            else:
                values.append("NA")

        # 각 컬럼에 맞춰 포맷팅하여 출력
        formatted_values = []
        formatted_values.append(f"{values[0]:^{widths['DateTime(YYYYMMDDHHMI)']}}")

        for i, col in enumerate(columns[1:], 1):
            formatted_values.append(f"{values[i]:^{widths[col]}}")

        print(" ".join(formatted_values))

Requesting data from 2020-01-14 00:00:00 to 2020-02-13 00:00:00
Requesting data from 2020-02-13 00:00:00 to 2020-03-14 00:00:00
Requesting data from 2020-03-14 00:00:00 to 2020-04-13 00:00:00
Requesting data from 2020-04-13 00:00:00 to 2020-05-13 00:00:00
Requesting data from 2020-05-13 00:00:00 to 2020-06-12 00:00:00
Requesting data from 2020-06-12 00:00:00 to 2020-06-14 00:00:00
DateTime(YYYYMMDDHHMI)  STN      TA       RN     RN_INT   RN_DAY    WW     WC     WP    SD_DAY   SD_TOT   SD_HR3   CA_TOT   CT_TOP   CT_LOW     WH      WD     WS      HM   
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    202001140000      108     -4.4     -9.0     -9.0     -9.0     -      -9     -9     -9.0     -9.0     -9.0      2        1        0       -9.0     34    0.5     46.0  
    202001140100      108     -4.4     -9.0     -9.0     -9.0     -      -9     -9     -9.0     -9.

# 새 섹션